## Created by <a href="https://github.com/yunsuxiaozi/">yunsuxiaozi</a> 2024/8/15

#### 这是2024年科大讯飞AI开发者大赛的电力需求预测挑战赛的top8 solution(8/1668)。



In [1]:
import pandas as pd#导入csv文件的库
import numpy as np#矩阵运算与科学计算的库
#model lgb回归模型,日志评估
from  lightgbm import LGBMRegressor,log_evaluation
from catboost import CatBoostRegressor
from tqdm import tqdm#导入进度条的库
import gc#垃圾回收模块
import warnings#避免一些可以忽略的报错
warnings.filterwarnings('ignore')#filterwarnings()方法是用于设置警告过滤器的方法，它可以控制警告信息的输出方式和级别。

In [2]:
#config
class Config():
    seed=2024#随机种子
    TARGET_NAME ='target'#标签
import random#提供了一些用于生成随机数的函数
#设置随机种子,保证模型可以复现
def seed_everything(seed):
    np.random.seed(seed)#numpy的随机种子
    random.seed(seed)#python内置的随机种子
seed_everything(Config.seed)

In [3]:
path='/kaggle/input/kedapower/2407keda/'
train=pd.read_csv(path+"train.csv")
#手动修正异常值
train.loc[(train['id']=='c3cef2b615')&(train['dt']==342),'target']=900#突然太低了,改高一点
train.loc[(train['id']=='5c3d91e81a')&(train['dt']==342),'target']=700#突然太低了,改高一点
train.loc[(train['id']=='df979f8f47')&(train['dt']==339),'target']=700#突然太低了,改高一点
test=pd.read_csv(path+"test.csv")
train.head()

,id,dt,type,target
0,00037f39cf,11,2,44.050
1,00037f39cf,12,2,50.672
2,00037f39cf,13,2,39.042
3,00037f39cf,14,2,35.900
4,00037f39cf,15,2,53.888


In [4]:
#趋势如何
IDS=train['id'].unique()
IDS_weight={}
for ID in tqdm(IDS):
    tmp=train[train['id']==ID]
    if len(tmp)>410:#用今年和去年的数据进行比较,看看趋势如何
        mean_last_year=np.mean(tmp[(tmp['dt']>=380)&(tmp['dt']<410)]['target'].values)
        mean_cur_year=np.mean(tmp[(tmp['dt']>=10)&(tmp['dt']<40)]['target'].values)
        scale=(1+mean_cur_year/mean_last_year)/2
        if scale<0.9125:
            scale=0.9125
        if scale>1.0875:
            scale=1.0875
        IDS_weight[ID]=scale
IDS_weight

100%|██████████| 5832/5832 [48:14<00:00,  2.01it/s]


{'00037f39cf': 0.9630469149595047,
 '00039a1517': 1.0130333242030387,
 '000c15d0ea': 1.0875,
 '00150bc11a': 1.0309675885700915,
 '0038d86077': 1.0354797131502986,
 '00431bea7c': 0.9789188703899332,
 '00476816e4': 1.0875,
 '005fc4185b': 0.9185451101535096,
 '00603386e3': 1.0085923089453788,
 '00629d85f8': 0.9899075343817011,
 '0068ddd8e4': 0.9843283233181033,
 '007d45c89d': 0.9125,
 '0087cf99a4': 0.9447340674102468,
 '0095046d02': 1.0244518220453145,
 '00a34678bb': 1.0875,
 '00b8f2fdf1': 1.042920077933537,
 '00bfd66f47': 1.056395377224725,
 '00c08d89fe': 0.9653007827684514,
 '00cc75b3c3': 1.0875,
 '00e2c609b4': 1.0227027188141276,
 '00e942bbdf': 0.9256014142011888,
 '00f6cfcec1': 1.0813985360346017,
 '00f7f5c26f': 0.9389707008181247,
 '010b9063e5': 0.9586983128169309,
 '011ba6714b': 0.9535387732854385,
 '012b946b10': 1.0875,
 '01375f5365': 0.9965142285287667,
 '014e9fb03e': 1.0051897972040096,
 '016562d2d3': 0.9125,
 '0169084125': 0.9125,
 '0175a63483': 0.9599065376466316,
 '017ae98719'

In [5]:
#10天前的数据
dt=pd.DataFrame({"dt":np.arange(11,507)})
dt['target_dt_mean']=train.groupby(['dt'])['target'].mean().values
dt['target_dt_min']=train.groupby(['dt'])['target'].min().values
dt['target_dt_max']=train.groupby(['dt'])['target'].max().values
dt['target_dt_std']=train.groupby(['dt'])['target'].std().values
dt['target_dt_median']=train.groupby(['dt'])['target'].median().values
dt['target_dt_skew']=train.groupby(['dt'])['target'].skew().values
for gap in [-1,-3,-6]:
    for col in ['target_dt_mean','target_dt_min','target_dt_max','target_dt_std','target_dt_median','target_dt_skew']:
        dt[f"{col}_shift{gap}"]=dt[col].shift(gap)    
        dt[f'{col}_gap{gap}']=dt[col]-dt[f"{col}_shift{gap}"]
        dt[f"{col}_diff{gap}"]=dt[col].diff(gap)  
dt.head()

,dt,target_dt_mean,target_dt_min,target_dt_max,target_dt_std,target_dt_median,target_dt_skew,target_dt_mean_shift-1,target_dt_mean_gap-1,target_dt_mean_diff-1,...,target_dt_max_diff-6,target_dt_std_shift-6,target_dt_std_gap-6,target_dt_std_diff-6,target_dt_median_shift-6,target_dt_median_gap-6,target_dt_median_diff-6,target_dt_skew_shift-6,target_dt_skew_gap-6,target_dt_skew_diff-6
0,11,33.642333,-0.179,1006.339,45.984458,25.3775,8.479711,34.708126,-1.065794,-1.065794,...,36.150,52.711293,-6.726835,-6.726835,24.6275,0.7500,0.7500,6.990756,1.488955,1.488955
1,12,34.708126,-0.350,988.884,49.556429,25.1670,7.896785,35.109676,-0.401550,-0.401550,...,20.667,45.204925,4.351505,4.351505,25.2565,-0.0895,-0.0895,8.229531,-0.332746,-0.332746
2,13,35.109676,-0.407,951.400,54.480135,23.9100,7.155603,35.204590,-0.094914,-0.094914,...,-92.253,48.686282,5.793853,5.793853,25.2470,-1.3370,-1.3370,7.845401,-0.689799,-0.689799
3,14,35.204590,-0.379,987.417,54.445576,23.8565,7.184043,35.447726,-0.243136,-0.243136,...,-14.391,53.967649,0.477927,0.477927,23.5505,0.3060,0.3060,7.299039,-0.114996,-0.114996
4,15,35.447726,-0.227,985.393,53.583230,24.1320,6.727281,36.212711,-0.764985,-0.764985,...,-141.434,54.598538,-1.015308,-1.015308,23.7635,0.3685,0.3685,7.390508,-0.663227,-0.663227


In [6]:
dt_type=train.groupby(['dt','type'])['target'].mean().reset_index().rename(columns={"target":'target_dt_type_mean'})
dt_type['target_dt_type_min']=train.groupby(['dt','type'])['target'].min().values
dt_type['target_dt_type_max']=train.groupby(['dt','type'])['target'].max().values
dt_type['target_dt_type_std']=train.groupby(['dt','type'])['target'].std().values
dt_type['target_dt_type_median']=train.groupby(['dt','type'])['target'].median().values
dt_type['target_dt_type_skew']=train.groupby(['dt','type'])['target'].skew().values
for gap in [-1,-2,-5]:
    for col in ['target_dt_type_mean','target_dt_type_min','target_dt_type_max','target_dt_type_std','target_dt_type_median','target_dt_type_skew'] :
        dt_type[f"{col}_shift{gap}"]=dt_type.groupby(['type'])[col].shift(gap)   
        dt_type[f"{col}_diff{gap}"]=dt_type.groupby(['type'])[col].diff(gap)   
        dt_type[f'{col}_gap{gap}']=dt_type[col]-dt_type[f"{col}_shift{gap}"]
dt_type.head()

,dt,type,target_dt_type_mean,target_dt_type_min,target_dt_type_max,target_dt_type_std,target_dt_type_median,target_dt_type_skew,target_dt_type_mean_shift-1,target_dt_type_mean_diff-1,...,target_dt_type_max_gap-5,target_dt_type_std_shift-5,target_dt_type_std_diff-5,target_dt_type_std_gap-5,target_dt_type_median_shift-5,target_dt_type_median_diff-5,target_dt_type_median_gap-5,target_dt_type_skew_shift-5,target_dt_type_skew_diff-5,target_dt_type_skew_gap-5
0,11,0,17.117879,-0.117,50.349,8.656841,16.135,0.578244,16.995354,0.122525,...,-1.169,7.953277,0.703564,0.703564,15.308,0.827,0.827,0.701482,-0.123238,-0.123238
1,11,1,18.912549,-0.054,68.946,11.831516,17.548,0.686096,18.599414,0.313135,...,-33.312,11.481577,0.349939,0.349939,16.697,0.851,0.851,1.302415,-0.616318,-0.616318
2,11,2,36.338097,-0.105,102.644,15.042624,35.479,0.454719,35.441517,0.896580,...,6.118,12.961026,2.081598,2.081598,33.198,2.281,2.281,0.733624,-0.278905,-0.278905
3,11,3,38.570670,0.031,100.441,16.874975,38.473,0.112511,39.889103,-1.318432,...,-3.675,15.121691,1.753283,1.753283,39.902,-1.429,-1.429,0.690091,-0.577580,-0.577580
4,11,4,42.583351,-0.166,158.699,24.155875,40.769,0.602351,41.617332,0.966019,...,41.578,22.166593,1.989282,1.989282,41.887,-1.118,-1.118,0.496720,0.105631,0.105631


In [7]:
total=pd.concat((train,test),axis=0)
#id从小到大,dt从大到小,因为dt是距离现在多少天.
total=total.sort_values(['id','dt'],ascending=[True,False])
print(f"len(total):{len(total)}")
total.head()

len(total):2935625


,id,dt,type,target
495,00037f39cf,506,2,33.141
494,00037f39cf,505,2,33.523
493,00037f39cf,504,2,29.933
492,00037f39cf,503,2,32.045
491,00037f39cf,502,2,40.008


In [8]:
print("type feature")
user_type=total['type'].unique()
for t in user_type:
    total[f'type_{t}']=(total['type']==t)
    
print("dt feature")  
#看到10天以前target的统计特征,gap特征
#看到10天前每个type的target的统计特征和gap特征
total['dt']=total['dt']+10
total=total.merge(dt,on='dt',how='left')
total=total.merge(dt_type,on=['dt','type'],how='left')
total['dt']=total['dt']-10

for day in [7,30,90,180,365]:
    total[f'sin_dt_{day}']=np.sin(2*np.pi*total['dt']/day)
    total[f'cos_dt_{day}']=np.cos(2*np.pi*total['dt']/day)

total['dt_dayofweek']=total['dt']%7
for d in range(7):
    total[f'dt_dayofweek_{d}']=(total['dt_dayofweek']==d)  
total['dt_month']=total['dt']//30%12
for d in range(12):
    total[f'dt_month_{d}']=(total['dt_month']==d)

type feature
dt feature


In [9]:
print("target feature")
for gap in [10,13,20]:
    for col in ['target']:
        total[f"{col}_shift{gap}"]=total.groupby(total['id'])[col].shift(gap) 
        for d in range(1,6):
            total[f'{col}_shift{gap}_diff{d}'] = total.groupby('id')[f'{col}_shift{gap}'].diff(d)
        total[f'{col}_shift{gap}_diff2-1']=total[f'{col}_shift{gap}_diff2']-total[f'{col}_shift{gap}_diff1']
        total[f'{col}_shift{gap}_diff3-1']=total[f'{col}_shift{gap}_diff3']-total[f'{col}_shift{gap}_diff1']
        total[f'{col}_shift{gap}_diff4-1']=total[f'{col}_shift{gap}_diff4']-total[f'{col}_shift{gap}_diff1']
        for w in [3,6,10]:
            total[f"{col}_gap{gap}_window{w}_mean"]=total.groupby(total['id'])[col].transform(lambda x: x.rolling(w).mean()).shift(gap)
            total[f"{col}_gap{gap}_window{w}_min"]=total.groupby(total['id'])[col].transform(lambda x: x.rolling(w).min()).shift(gap)
            total[f"{col}_gap{gap}_window{w}_max"]=total.groupby(total['id'])[col].transform(lambda x: x.rolling(w).max()).shift(gap)
            total[f"{col}_gap{gap}_window{w}_std"]=total.groupby(total['id'])[col].transform(lambda x: x.rolling(w).std()).shift(gap)
            total[f"{col}_gap{gap}_window{w}_median"]=total.groupby(total['id'])[col].transform(lambda x: x.rolling(w).median()).shift(gap)
            total[f"{col}_gap{gap}_window{w}_skew"]=total.groupby(total['id'])[col].transform(lambda x: x.rolling(w).skew()).shift(gap)   
            
for col in [['id'],['type'],['dt_dayofweek'],['dt_month'],
            ['id','dt_dayofweek'],['id','dt_month'],
            ['type','dt_dayofweek'],['type','dt_month']
           ]:
    print(f"col:{col}")
    if len(col)>1:
        colname='_'.join(col)
    else:
        colname=col[0]
    total[f"{colname}_target_min"]=total.groupby(col)['target'].transform('min')
    total[f"{colname}_target_max"]=total.groupby(col)['target'].transform('max')
    total[f"{colname}_target_mean"]=total.groupby(col)['target'].transform('mean')
    total[f"{colname}_target_std"]=total.groupby(col)['target'].transform('std')
    total[f"{colname}_target_skew"]=total.groupby(col)['target'].transform('skew')
    total[f"{colname}_target_last"]=total.groupby(col)['target'].transform('last')
    total[f"{colname}_target_first"]=total.groupby(col)['target'].transform('first')
    
    total[f"{colname}_target_ptp"]=total[f"{colname}_target_max"]-total[f"{colname}_target_min"]
    total[f"{colname}_target_max/min"]=total[f"{colname}_target_max"]/(total[f"{colname}_target_min"]+1e-15)
    total[f"{colname}_target_mean/std"]=total[f"{colname}_target_mean"]/(total[f"{colname}_target_std"]+1e-15)

target feature
col:['id']
col:['type']
col:['dt_dayofweek']
col:['dt_month']
col:['id', 'dt_dayofweek']
col:['id', 'dt_month']
col:['type', 'dt_dayofweek']
col:['type', 'dt_month']


In [10]:
train=total[~total['target'].isna()]
test=total[total['target'].isna()]
del total
gc.collect()
print(f"features_count:{len(test.columns)}")
train.head()

features_count:335


,id,dt,type,target,type_2,type_4,type_0,type_5,type_1,type_3,...,type_dt_month_target_min,type_dt_month_target_max,type_dt_month_target_mean,type_dt_month_target_std,type_dt_month_target_skew,type_dt_month_target_last,type_dt_month_target_first,type_dt_month_target_ptp,type_dt_month_target_max/min,type_dt_month_target_mean/std
0,00037f39cf,506,2,33.141,True,False,False,False,False,False,...,-0.504,111.415,27.937425,12.541604,0.592792,13.417,33.141,111.919,-221.061508,2.22758
1,00037f39cf,505,2,33.523,True,False,False,False,False,False,...,-0.504,111.415,27.937425,12.541604,0.592792,13.417,33.141,111.919,-221.061508,2.22758
2,00037f39cf,504,2,29.933,True,False,False,False,False,False,...,-0.504,111.415,27.937425,12.541604,0.592792,13.417,33.141,111.919,-221.061508,2.22758
3,00037f39cf,503,2,32.045,True,False,False,False,False,False,...,-0.504,111.415,27.937425,12.541604,0.592792,13.417,33.141,111.919,-221.061508,2.22758
4,00037f39cf,502,2,40.008,True,False,False,False,False,False,...,-0.504,111.415,27.937425,12.541604,0.592792,13.417,33.141,111.919,-221.061508,2.22758


In [11]:
#遍历表格df的所有列修改数据类型减少内存使用
def reduce_mem_usage(df, float16_as32=True):
    #memory_usage()是df每列的内存使用量,sum是对它们求和, B->KB->MB
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:#遍历每列的列名
        col_type = df[col].dtype#列名的type
        if col_type != object and str(col_type)!='category':#不是object也就是说这里处理的是数值类型的变量
            c_min,c_max = df[col].min(),df[col].max() #求出这列的最大值和最小值
            if str(col_type)[:3] == 'int':#如果是int类型的变量,不管是int8,int16,int32还是int64
                #如果这列的取值范围是在int8的取值范围内,那就对类型进行转换 (-128 到 127)
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                #如果这列的取值范围是在int16的取值范围内,那就对类型进行转换(-32,768 到 32,767)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                #如果这列的取值范围是在int32的取值范围内,那就对类型进行转换(-2,147,483,648到2,147,483,647)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                #如果这列的取值范围是在int64的取值范围内,那就对类型进行转换(-9,223,372,036,854,775,808到9,223,372,036,854,775,807)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:#如果是浮点数类型.
                #如果数值在float16的取值范围内,如果觉得需要更高精度可以考虑float32
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    if float16_as32:#如果数据需要更高的精度可以选择float32
                        df[col] = df[col].astype(np.float32)
                    else:
                        df[col] = df[col].astype(np.float16)  
                #如果数值在float32的取值范围内，对它进行类型转换
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                #如果数值在float64的取值范围内，对它进行类型转换
                else:
                    df[col] = df[col].astype(np.float64)
    #计算一下结束后的内存
    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    #相比一开始的内存减少了百分之多少
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df
train=reduce_mem_usage(train, float16_as32=False)
test=reduce_mem_usage(test, float16_as32=False)

Memory usage of dataframe is 6646.00 MB
Memory usage after optimization is: 1885.14 MB
Decreased by 71.6%
Memory usage of dataframe is 134.71 MB
Memory usage after optimization is: 38.38 MB
Decreased by 71.5%


In [12]:
for col in test.drop(['id'],axis=1).columns:
    skew=train[col].skew()
    if abs(skew)>0.5:
        min_value=train[col].min()
        train[col]=train[col]-min_value
        test[col]=test[col]-min_value
        
        print(f"col:{col},skew:{skew}")
        train[col]=np.log1p(train[col])
        test[col]=np.log1p(test[col])

col:type,skew:1.250220916500952
col:id_dt_dayofweek_target_max/min,skew:inf
col:id_dt_month_target_max/min,skew:inf


In [13]:
choose_cols=[ col for col in test.drop(['id','target'],axis=1).columns]

lgb_params = { "boosting_type": "gbdt","objective": "regression","metric": "mse",       
               'random_state': 2024, 'n_estimators':1024,
                'reg_alpha': 0.1, 'reg_lambda': 10, 
                'colsample_bytree': 0.8, 'subsample': 0.8,
                'learning_rate': 0.05, 'num_leaves': 64, 'min_child_samples': 62,
                'max_bin':255, "extra_trees": True,
                'device':'gpu','gpu_use_dp':True,#这行GPU环境的参数,想在CPU环境下运行注释这行代码
             }

X=train[choose_cols].copy()
y=train[Config.TARGET_NAME].copy()
test_X=test[choose_cols].copy()

model=LGBMRegressor(**lgb_params)
model.fit(X,y)
test_preds1=model.predict(test_X)
del model
gc.collect()


cat_params = {'learning_rate': 0.1, 'depth': 5, 'bootstrap_type':'Bernoulli',
          'thread_count':-1, 'od_type': 'Iter', 'od_wait': 100, 
          'random_seed': 2024, 'allow_writing_files': False,
          'task_type':'GPU'
         }
model = CatBoostRegressor(iterations=1034, **cat_params)
model.fit(X,y,verbose=1)
test_preds2 = model.predict(test_X)
del model
gc.collect()

test_preds=(0.7*test_preds1+0.3*test_preds2)

print(test_preds[:10])

[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 63519
[LightGBM] [Info] Number of data points in the train set: 2877305, number of used features: 333
[LightGBM] [Info] Using GPU Device: Tesla P100-PCIE-16GB, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...


1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.


[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 16
[LightGBM] [Info] 298 dense feature groups (823.20 MB) transferred to GPU in 0.499683 secs. 0 sparse feature groups
[LightGBM] [Info] Start training from score 32.218119
0:	learn: 46.2274499	total: 24.7s	remaining: 7h 4m 38s
1:	learn: 42.5544480	total: 24.7s	remaining: 3h 32m 19s
2:	learn: 39.1909596	total: 24.7s	remaining: 2h 21m 34s
3:	learn: 36.1716486	total: 24.7s	remaining: 1h 46m 10s
4:	learn: 33.5196475	total: 24.8s	remaining: 1h 24m 56s
5:	learn: 31.1261466	total: 24.8s	remaining: 1h 10m 47s
6:	learn: 29.0332082	total: 24.8s	remaining: 1h 41s
7:	learn: 27.1334865	total: 24.8s	remaining: 53m 6s
8:	learn: 25.4788864	total: 24.9s	remaining: 47m 12s
9:	learn: 24.0041419	total: 24.9s	remaining: 42m 29s
10:	learn: 22.7165354	total: 24.9s	remaining: 38m 37s
11:	learn: 21.5843153	total: 24.9s	remaining: 35m 24s
12:	learn: 20.6009218	total: 25s	remaining: 32m 41s
13:	learn: 19.7139428	total:

In [14]:
submission=test[['id','dt']]
submission['target']=test_preds
submission=submission.sort_values(['id','dt'])
print("fine_tune")
for ID,weight in IDS_weight.items():
    submission.loc[(submission['id'].str.contains(ID)),'target']*=weight

submission.to_csv("top8solution.csv",index=None)
submission.head()

fine_tune


,id,dt,target
505,00037f39cf,1,45.024816
504,00037f39cf,2,39.951437
503,00037f39cf,3,39.884842
502,00037f39cf,4,39.203083
501,00037f39cf,5,40.531135
